In [1]:
from pyprojroot import here
import duckdb

In [7]:

raw_folder = here() / "src" / "raw"
src_folder = here() / "src"
query_folder =  here() / "src" / "query"
processed_folder = here() / "src" / "processed"
scale_factor = 5

print(raw_folder)

/workspace/src/raw


In [8]:
# Create folders if they don't exist
for folder in [raw_folder, query_folder, processed_folder]:
    if not folder.exists():
        print(f"Creating {folder} directory...")
        folder.mkdir(parents=True, exist_ok=True)
    else:
        print(f"{folder} directory already exists")


/workspace/src/raw directory already exists
Creating /workspace/src/query directory...
Creating /workspace/src/processed directory...


In [9]:
# Create duckdb database
con = duckdb.connect()

In [4]:

# Generate TPC-H data
con.execute("INSTALL tpch; LOAD tpch;")
con.execute(f"CALL dbgen(sf={scale_factor});")  # sf = scale factor

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [5]:
tables = ["nation", "region", "part", "supplier", "partsupp", "customer", "orders", "lineitem"]

(raw_folder / f"{scale_factor:02d}").mkdir(parents=True, exist_ok=True)

for table in tables:
    path = raw_folder / f"{scale_factor:02d}" / f"{table}.parquet"
    con.execute(f"COPY {table} TO '{path}' (FORMAT 'parquet');")

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [10]:
con = duckdb.connect()
queries = con.execute("SELECT * FROM tpch_queries()").fetchall()

for i, row in enumerate(queries, 1):
    sql = row[1]
    (query_folder / f"tpch_q{i:02}.sql").write_text(sql)

con.close()